# Setup Code

Install necessary libraries and clone my project

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!git clone https://github.com/jihwangk/NLP.git
!pip install transformers
!pip install tensorboardX
# !pip install scikit-learn
%cd NLP/project/

# # Since annotations from __future__ module is in Python 3.7 and after, update python version (choose the manual 3.7 version if prompted)
# !python --version
# !sudo apt-get install python3.7
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 1
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 2
# !sudo update-alternatives --config python3
# !python --version

Cloning into 'NLP'...
remote: Enumerating objects: 1926, done.
remote: Counting objects: 100% (1926/1926), done.
remote: Compressing objects: 100% (1823/1823), done.
remote: Total 1926 (delta 29), reused 1916 (delta 19), pack-reused 0
Receiving objects: 100% (1926/1926), 16.86 MiB | 7.75 MiB/s, done.
Resolving deltas: 100% (29/29), done.
     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 45.4MB/s 
     |████████████████████████████████| 1.0MB 35.1MB/s 
     |████████████████████████████████| 890kB 40.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=aa11db923c32859147f80aabc009fd189e842c28cd2b38fbce74e7e5da5435fb
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
     |████████████████████████████████| 204kB 2.8MB/s 
/content/NLP/project


Manage all imports for this project here

In [0]:
import os
import glob
import codecs
import csv
import pandas
import logging
import math
import numpy
import pickle
from collections import defaultdict, Counter
from multiprocessing import Pool, cpu_count
from tqdm import tqdm, tqdm_notebook, trange

import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tensorboardX import SummaryWriter

from transformers import (BertConfig, BertTokenizer, BertForSequenceClassification,
                          WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup)

from sklearn.metrics import f1_score

Define folders to get the dataset needed

In [0]:
train_articles = "datasets/train-articles"
dev_articles = "datasets/dev-articles"
test_articles = "test/test-articles"

train_SI_labels = "datasets/train-labels-task1-span-identification"
train_TC_labels = "datasets/train-labels-task2-technique-classification"
dev_SI_labels = "gold_labels/dev-labels-task1-span-identification"
dev_TC_labels = "gold_labels/dev-labels-task2-technique-classification"
dev_TC_labels_file = "gold_labels/dev-task-TC.labels"

dev_TC_template = "datasets/dev-task-TC-template.out"
test_TC_template = "test/test-task-TC-template.out"

techniques = "tools/data/propaganda-techniques-names-semeval2020task11.txt"
PROP_TECH_TO_LABEL = {}
LABEL_TO_PROP_TECH = {}
label = 0
with open(techniques, "r") as f:
  for technique in f:
    PROP_TECH_TO_LABEL[technique.replace("\n", "")] = int(label)
    LABEL_TO_PROP_TECH[int(label)] = technique.replace("\n", "")
    label += 1

In [0]:
# count = defaultdict(int)
# with open("gold_labels/dev-task-TC.labels", "r") as f:
#   reader = csv.reader(f, delimiter="\t")
#   for row in reader:
#     count[row[1]] += 1
# print(count)

defaultdict(<class 'int'>, {'Whataboutism,Straw_Men,Red_Herring': 29, 'Flag-Waving': 87, 'Loaded_Language': 325, 'Causal_Oversimplification': 18, 'Appeal_to_Authority': 14, 'Doubt': 66, 'Appeal_to_fear-prejudice': 44, 'Exaggeration,Minimisation': 68, 'Name_Calling,Labeling': 183, 'Slogans': 40, 'Thought-terminating_Cliches': 17, 'Bandwagon,Reductio_ad_hitlerum': 5, 'Black-and-White_Fallacy': 22, 'Repetition': 145})


In [0]:
# output = {}
# with codecs.open("baseline-output-TC.txt", "r", encoding="utf8") as f:
#     reader = csv.reader(f, delimiter="\t")
#     for row in reader:
#       output[tuple([row[0], row[2], row[3]])] = row[1]

# with open("/../../../content/gdrive/My Drive/SI630/Project/results/TC_baseline_lin_reg.txt", "w") as f_total:
#   writer = csv.writer(f_total, delimiter="\t")

#   with codecs.open(dev_TC_template, "r", encoding="utf8") as f:
#     reader = csv.reader(f, delimiter="\t")
#     for row in reader:
#       key_1 = tuple([row[0], row[2], row[3]])
#       key_2 = tuple([row[0], row[2], str(int(row[3]) + 1)])
#       if key_1 in output.keys():
#         writer.writerow([row[0], output[key_1], row[2], row[3]])
#       elif key_2 in output.keys():
#         writer.writerow([row[0], output[key_2], row[2], row[3]]) 
#       else:
#         writer.writerow([row[0], "Loaded_Language", row[2], row[3]])

Finish the setup with initializing configs

In [0]:
device = torch.device("cuda")
# device = torch.device("cpu")
n_gpu = torch.cuda.device_count()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("LOG")

Preprocess the given articles into readable data by BERT

In [0]:
def save_to_pickle(dataset, fileloc):
  """
  Saves dataset to a pickle to specified location
  """
  filename = fileloc + ".pickle"
  with open(filename, 'wb') as f:
    pickle.dump(dataset, f)
  logger.info("Saved to {}".format(filename))

def load_from_pickle(fileloc):
  """
  Loads dataset from a pickle
  """
  filename = fileloc + ".pickle"
  with open(filename, 'rb') as f:
    dataset = pickle.load(f)
  logger.info("Loaded from {}".format(filename))
  return dataset
  
def detokenize(tokenized, tokenizer):
  """
  Tries to detokenize tokenized sequence
  """
  # For each tokens, do:
  for i in range(len(tokenized)):
    # If the token is one of the following, delete space
    if  tokenized[i] in "”’)]}>.,?!:;-_":
      tokenized[i] = "##" + tokenized[i]

    # If current token is not the last, and
    if i != len(tokenized) - 1:
      # If current token is one of the following, delete space in following token
      if tokenized[i] in "“([{<-_" or (tokenized[i] == "’" and tokenized[i + 1] == "s"):
        tokenized[i + 1] = "##" + tokenized[i + 1]
  
  # Revert into string
  reverted = tokenizer.convert_tokens_to_string(tokenized)
  return reverted


def merge_overlapping(indices_list):
  """
  Merges overlapping indices and sorts indices from list of tuples
  """
  # If no propaganda, return empty list
  if indices_list == []:
    return []

  # Sort the list
  indices_list = sorted(indices_list)
  i = 0

  # Going through tuples from the beginning, see if it overlaps with the nex one 
  # and merge it if so
  while True:
    if i == len(indices_list) - 1:
      break
    
    # If the next one is within the range of current, just delete next one. If
    # overlapping, then merge the range
    elif indices_list[i][1] >= indices_list[i + 1][0]:
      if indices_list[i][1] >= indices_list[i + 1][1]:
        pass
      else:
        indices_list[i] = (indices_list[i][0], indices_list[i+1][1])
      indices_list.pop(i+1)

    # Go to next element if not overlapping
    else:
      i += 1

  return indices_list

def article_to_sequences(article_id, article, tokenizer):
  """
  Divides article into sequences, dividing first by sentences then to powersets
  of the sentences
  """
  # Split the lines by sentences
  curr = 0
  lines = article.split("\n")
  sequences = []
  seq_starts = []
  seq_ends = []

  # For each lines, do:
  for line in lines:
    # If an empty line, just continue
    if line == "":
      curr += 1
      continue

    # Tokenize the line
    tokenized = tokenizer.tokenize(line)

    # For each token, do:
    seq_start = 0
    for ind, token in enumerate(tokenized):
      # Get the token without ## sign
      mod_start_token = token.replace("##", "")

      # Find the start of the sequence in line
      seq_start = line.lower().find(mod_start_token, seq_start)

      # Update the end of the sequence
      seq_end = seq_start

      # For each following tokens in the line, do
      for iter in range(1, len(tokenized) + 1 - ind):
        # Also modify this token
        mod_end_token = tokenized[ind + iter - 1].replace("##", "")
        # Find the end of the token
        seq_end = line.lower().find(mod_end_token, seq_end) + len(mod_end_token)

        sequences.append(tokenizer.convert_tokens_to_string(tokenized[ind: ind + iter]))
        seq_starts.append(curr + seq_start)
        seq_ends.append(curr + seq_end)

      # Update the start of the sequence
      seq_start += len(mod_start_token)

    # Update the current whereabouts
    curr += len(line) + 1

  dataframe = pandas.DataFrame(None, range(len(sequences)), ["id", "seq_starts", "seq_ends", "label", "text"])
  dataframe["id"] = [article_id] * len(sequences)
  dataframe["seq_starts"] = seq_starts
  dataframe["seq_ends"] = seq_ends
  dataframe["label"] = [0] * len(sequences)
  dataframe["text"] = sequences
  return dataframe

def article_labels_to_sequences(article, indices_list):
  """
  Divides article into sequences, where each are tagged to be propaganda or not
  """
  # Start at 0 indices, and split the article into lines
  curr = 0
  lines = article.split("\n")
  sequences = {}

  # For each lines, do:
  for line in lines:
    # If an empty line, just continue after adding \n character
    if line == "":
      curr += 1
      continue

    # If nothing in indices_list or current line is not part of propaganda, 
    # just mark it to be none 
    elif indices_list == [] or curr + len(line) <= indices_list[0][0]:
      sequences[line] = 0

    # If current line is part of propaganda, do:
    else:
      # If the propaganda is contained within the line, add it accordingly
      # and pop that indices range
      if curr + len(line) >= indices_list[0][1]:
        sequences[line[:indices_list[0][0] - curr]] = 0
        sequences[line[indices_list[0][0] - curr:indices_list[0][1] - curr]] = 1
        sequences[line[indices_list[0][1] - curr:]] = 0
        indices_list.pop(0)
      # If the propaganda goes over to the next line, add accordingly and 
      # modify that indices range
      else:
        sequences[line[:indices_list[0][0] - curr]] = 0
        sequences[line[indices_list[0][0] - curr:]] = 1
        indices_list[0][0] = curr + len(line) + 2

    # Add the current line length plus \n character
    curr += len(line) + 1

  dataframe = pandas.DataFrame(None, range(len(sequences)), ["label", "text"])
  dataframe["label"] = sequences.values()
  dataframe["text"] = sequences.keys()
  return dataframe

def articles_to_dataframe(article_folder, label_folder, task="SI"):
  """
  Preprocesses the articles into dataframes with sequences with binary tags
  """
  # First sort the filenames and make sure we have label file for each articles
  article_filenames = sorted(glob.glob(os.path.join(article_folder, "*.txt")))
  label_filenames = sorted(glob.glob(os.path.join(label_folder, "*.labels")))
  assert len(article_filenames) == len(label_filenames)

  # Initialize sequences
  sequences = []

  # For each article, do:
  for i in range(len(article_filenames)):
    # Get the id name
    article_id = os.path.basename(article_filenames[i]).split(".")[0][7:]

    # Read in the article
    with codecs.open(article_filenames[i], "r", encoding="utf8") as f:
      article = f.read()

    # Read in the label file and store indices for SI task
    if task == "SI":
      with open(label_filenames[i], "r") as f:
        reader = csv.reader(f, delimiter="\t")
        indices_list = []
        for row in reader:
          indices_list.append([int(row[1]), int(row[2])])

        # Merge the indices if overlapping
        indices_list = merge_overlapping(indices_list)

      # Add to the sequences
      sequences.append(article_labels_to_sequences(article, indices_list))

    # Read in the label file and store indices for TC task
    elif task == "TC":
      with open(label_filenames[i], "r") as f:
        reader = csv.reader(f, delimiter="\t")
        article_sequences = []
        labels_list = []
        for row in reader:
          article_sequences.append(article[int(row[2]):int(row[3])])
          labels_list.append(PROP_TECH_TO_LABEL[row[1]])

      sequence = pandas.DataFrame(None, range(len(article_sequences)), ["label", "text"])
      sequence["label"] = labels_list
      sequence["text"] = article_sequences

      # Add to the sequences
      sequences.append(sequence)

    else:
      logger.error("Undefined task %s !", task)

  # Concatenate all dataframes
  dataframe = pandas.concat(sequences, ignore_index=True)

  return dataframe

def convert_dataframe_to_features(dataframe, max_seq_length, tokenizer):
  """
  Converts dataframe into features dataframe, where each feature will
  take form of [CLS] + A + [SEP]
  """
  # Create features
  features = pandas.DataFrame(None, range(dataframe.shape[0]), 
                              ["input_ids", "input_mask", "segment_ids", "label_ids"])

  # For each sequence, do:
  for i in range(len(dataframe)):
    # Set first and second part of the sequences
    tokens = tokenizer.tokenize(dataframe["text"][i])

    # If length of the sequence is greater than max sequence length, truncate it
    if len(tokens) > max_seq_length - 2:
        tokens = tokens[:(max_seq_length - 2)]

    # Concatenate the tokens
    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]

    # Compute the ids
    segment_ids = [0] * len(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    pad_token = tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0]
    input_ids = input_ids + [pad_token] * padding_length
    input_mask = input_mask + [0] * padding_length
    segment_ids = segment_ids + [0] * padding_length
    label_id = dataframe["label"][i]

    # Assert to make sure we have same length
    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    # Put the data into features dataframe
    features["input_ids"][i] = input_ids
    features["input_mask"][i] = input_mask
    features["segment_ids"][i] = segment_ids
    features["label_ids"][i] = label_id

  return features

def generate_training_dataset_from_articles(articles_folders, labels_folders, tokenizer, task="SI"):
  """
  Generates dataset to go into BERT from articles and labels
  """
  # If generating dataset for evaluation, do:
  logger.info("Generating training dataset...")
    
  # For each articles and labels folder set, turn them into dataframes
  dataframe_list = []
  for i in range(len(articles_folders)):
    logger.info("Generating dataframe for folder %s", articles_folders[i])
    dataframe_list.append(articles_to_dataframe(articles_folders[i], labels_folders[i], task=task))

  # Concatenate the dataframes to make a total dataframe
  dataframe = pandas.concat(dataframe_list, ignore_index=True)

  print(dataframe)
  print(dataframe.shape)

  # Process into features dataframe
  logger.info("Creating features from dataframe")
  features = convert_dataframe_to_features(dataframe, args['max_seq_length'], tokenizer)
      
  # Creating TensorDataset from features
  logger.info("Creating TensorDataset from features dataframe")
  all_input_ids = torch.tensor(features["input_ids"], dtype=torch.long)
  all_input_mask = torch.tensor(features["input_mask"], dtype=torch.long)
  all_segment_ids = torch.tensor(features["segment_ids"], dtype=torch.long)
  all_label_ids = torch.tensor(features["label_ids"], dtype=torch.long)

  dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
  return dataset

def generate_SI_eval_dataset_from_article(article_file, tokenizer):
  """
  Generates SI dataset to go into BERT from articles and labels
  """
  # If generating dataset for evaluation, do:
  logger.info("Generating evaluation dataset...")

  # Get the id name
  article_id = os.path.basename(article_file).split(".")[0][7:]

  # Read in the article
  with codecs.open(article_file, "r", encoding="utf8") as f:
    article = f.read()

  # For evaluating, do:
  dataframe = article_to_sequences(article_id, article, tokenizer)

  # Only include sequences that has alphanumeric characters in them
  bool_indices = [False] * len(dataframe)
  for i in range(len(dataframe)):
    if any(c.isalnum() for c in dataframe["text"][i]):
      bool_indices[i] = True
  dataframe = dataframe.loc[bool_indices].reset_index(drop=True)

  print(dataframe)
  print(dataframe.shape)

  # Process into features dataframe
  logger.info("Creating features from dataframe")
  features = convert_dataframe_to_features(dataframe, args['max_seq_length'], tokenizer)
      
  # Creating TensorDataset from features
  logger.info("Creating TensorDataset from features dataframe")
  all_input_ids = torch.tensor(features["input_ids"], dtype=torch.long)
  all_input_mask = torch.tensor(features["input_mask"], dtype=torch.long)
  all_segment_ids = torch.tensor(features["segment_ids"], dtype=torch.long)
  all_label_ids = torch.tensor(features["label_ids"], dtype=torch.long)

  dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
  return dataset, dataframe

def generate_TC_eval_dataset_from_article(article_folder, indices_file, tokenizer):
  """
  Generates TC dataset to go into BERT from articles and labels
  """
  # If generating dataset for evaluation, do:
  logger.info("Generating evaluation dataset...")

  # First sort the filenames and make sure we have label file for each articles
  article_filenames = sorted(glob.glob(os.path.join(article_folder, "*.txt")))
  articles = {}

  # For each article, read them in:
  for i in range(len(article_filenames)):
    article_id = os.path.basename(article_filenames[i]).split(".")[0][7:]
    with codecs.open(article_filenames[i], "r", encoding="utf8") as f:
      articles[article_id] = f.read()

  # Read in indices file
  with open(indices_file, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    ids_list = []
    seq_starts = []
    seq_ends = []
    article_sequences = []
    for row in reader:
      ids_list.append(row[0])
      seq_starts.append(row[2])
      seq_ends.append(row[3])
      article_sequences.append(articles[row[0]][int(row[2]):int(row[3])])

  dataframe = pandas.DataFrame(None, range(len(ids_list)), ["id", "seq_starts", "seq_ends", "label", "text"])
  dataframe["id"] = ids_list
  dataframe["seq_starts"] = seq_starts
  dataframe["seq_ends"] = seq_ends
  dataframe["label"] = [0] * len(ids_list)
  dataframe["text"] = article_sequences

  print(dataframe)
  print(dataframe.shape)

  # Process into features dataframe
  logger.info("Creating features from dataframe")
  features = convert_dataframe_to_features(dataframe, args['max_seq_length'], tokenizer)
      
  # Creating TensorDataset from features
  logger.info("Creating TensorDataset from features dataframe")
  all_input_ids = torch.tensor(features["input_ids"], dtype=torch.long)
  all_input_mask = torch.tensor(features["input_mask"], dtype=torch.long)
  all_segment_ids = torch.tensor(features["segment_ids"], dtype=torch.long)
  all_label_ids = torch.tensor(features["label_ids"], dtype=torch.long)

  dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
  return dataset, dataframe

# Task SI: Binary Sequence Tagging with BERT

## Preprocessing

Save preprocessed datasets

In [0]:
# Dictionary for different models
MODEL_CLASSES = {"bert": (BertConfig, BertForSequenceClassification, BertTokenizer)}

# Set arguments for training here
args = {"data_dir": "datasets/",
        "model_type": "bert",
        "model_name": "bert-base-uncased",
        "output_dir": "/../../../content/gdrive/My Drive/SI630/Project/models", # Change this path for your own
        "max_seq_length": 128,
        "train_batch_size": 8,
        "eval_batch_size": 8,
        "num_train_epochs": 5,
        "weight_decay": 0,
        "learning_rate": 5e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "gradient_accumulation_steps": 1,
        "logging_steps": 50,
        "save_steps": 2000,
        "overwrite_output_dir": False}

# # Preprocess tensor datasets, uncomment when files need to be generated. Path may need to be changed
# _, _, tokenizer_class = MODEL_CLASSES[args["model_type"]]
# tokenizer = tokenizer_class.from_pretrained(args["model_name"])

# train_dataset = generate_training_dataset_from_articles([train_articles], [train_SI_labels], tokenizer)
# save_to_pickle(train_dataset, "/../../../content/gdrive/My Drive/SI630/Project/datasets/SI/train-train")

# train_dataset = generate_training_dataset_from_articles([train_articles, dev_articles], [train_SI_labels, dev_SI_labels], tokenizer)
# save_to_pickle(train_dataset, "/../../../content/gdrive/My Drive/SI630/Project/datasets/SI/train-train-dev")

## Training

In [0]:
def train(train_dataset, model, tokenizer):
  """
  Trains the model with training dataset
  """
  # Initialize various necessary objects
  tb_writer = SummaryWriter()
  train_sampler = RandomSampler(train_dataset)
  train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])
    
  # Compute the total time
  t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
  
  # Set the grouped parameters for optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
  
  # Compute warmup steps
  warmup_steps = math.ceil(t_total * args['warmup_ratio'])
  args['warmup_steps'] = warmup_steps if args['warmup_steps'] == 0 else args['warmup_steps']
  
  # Initialize optimizer as Adam with constant weight decay and a linear scheduler with warmup
  optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args['warmup_steps'], num_training_steps=t_total)

  # Initialize variables for training
  global_step = 0
  tr_loss, logging_loss = 0.0, 0.0
  model.zero_grad()
  train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
  
  # Start training!
  for _ in train_iterator:
    epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
      model.train()
      batch = tuple(t.to(device) for t in batch)
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2], 
                'labels':         batch[3]}
      outputs = model(**inputs)
      loss = outputs[0]

      if args['gradient_accumulation_steps'] > 1:
        loss = loss / args['gradient_accumulation_steps']
          
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])

      tr_loss += loss.item()
      if (step + 1) % args['gradient_accumulation_steps'] == 0:
        optimizer.step()
        scheduler.step() 
        model.zero_grad()
        global_step += 1

        if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0:
          tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
          tb_writer.add_scalar('loss', (tr_loss - logging_loss)/args['logging_steps'], global_step)
          logging_loss = tr_loss

        if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
          output_dir = os.path.join(args['output_dir'], 'checkpoint-{}'.format(global_step))
          if not os.path.exists(output_dir):
            os.makedirs(output_dir)
          model_to_save = model.module if hasattr(model, 'module') else model
          model_to_save.save_pretrained(output_dir)
          logger.info("Saving model checkpoint to %s", output_dir)

  return global_step, tr_loss / global_step

In [0]:
# Get the classes and pretrained config, tokenizer, and model with predetermined model
config_class, model_class, tokenizer_class = MODEL_CLASSES[args["model_type"]]
config = config_class.from_pretrained(args["model_name"], num_labels=2, finetuning_task="binary")
tokenizer = tokenizer_class.from_pretrained(args["model_name"])
model = model_class(config)
model.to(device)

# Load preprocessed tensor dataset
train_dataset = load_from_pickle("/../../../content/gdrive/My Drive/SI630/Project/datasets/SI/train-train")
args['output_dir'] = "/../../../content/gdrive/My Drive/SI630/Project/models/SI/train"

# Train the model with the dataset
global_step, tr_loss = train(train_dataset, model, tokenizer)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)
logger.info("Saving model checkpoint to %s", args['output_dir'])
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(args['output_dir'])
tokenizer.save_pretrained(args['output_dir'])
torch.save(args, os.path.join(args['output_dir'], 'training_args.bin'))

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": "binary",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_e

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
INFO:transformers.configuration_utils:Configuration saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-2000/config.json
INFO:transformers.modeling_utils:Model weights saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-2000/pytorch_model.bin
INFO:LOG:Saving model checkpoint to /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-2000


Epoch:  20%|██        | 1/5 [20:37<1:22:29, 1237.32s/it]

INFO:transformers.configuration_utils:Configuration saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-4000/config.json
INFO:transformers.modeling_utils:Model weights saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-4000/pytorch_model.bin
INFO:LOG:Saving model checkpoint to /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-4000


Epoch:  40%|████      | 2/5 [41:05<1:01:44, 1234.72s/it]

INFO:transformers.configuration_utils:Configuration saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-6000/config.json
INFO:transformers.modeling_utils:Model weights saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-6000/pytorch_model.bin
INFO:LOG:Saving model checkpoint to /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-6000
INFO:transformers.configuration_utils:Configuration saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-8000/config.json
INFO:transformers.modeling_utils:Model weights saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-8000/pytorch_model.bin
INFO:LOG:Saving model checkpoint to /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-8000


Epoch:  60%|██████    | 3/5 [1:01:36<41:06, 1233.37s/it]

INFO:transformers.configuration_utils:Configuration saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-10000/config.json
INFO:transformers.modeling_utils:Model weights saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-10000/pytorch_model.bin
INFO:LOG:Saving model checkpoint to /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-10000


Epoch:  80%|████████  | 4/5 [1:22:01<20:30, 1230.80s/it]

INFO:transformers.configuration_utils:Configuration saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-12000/config.json
INFO:transformers.modeling_utils:Model weights saved in /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-12000/pytorch_model.bin
INFO:LOG:Saving model checkpoint to /../../../content/gdrive/My Drive/SI630/Project/models/SI/train/checkpoint-12000


##Classification

In [0]:
def classify_per_article(eval_dataframe, eval_dataset, model, tokenizer):
  """
  Classifies a single article dataset and returns article id with indices list
  """
  # Load the eval data and initialize sampler
  eval_sampler = SequentialSampler(eval_dataset)
  eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

  # Start Classification
  preds = None

  # For each batch, evaluate
  for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2],
                'labels':         batch[3]}
      outputs = model(**inputs)
      logits = outputs[1]

    # Get predictions
    if preds is None:
      preds = logits.detach().cpu().numpy()
    else:
      preds = numpy.append(preds, logits.detach().cpu().numpy(), axis=0)

  # Get the most probable prediction
  preds = numpy.argmax(preds, axis=1)
  eval_dataframe["label"] = preds

  # Get the propaganda indices and return them
  eval_dataframe = eval_dataframe.loc[eval_dataframe["label"] == 1].reset_index(drop=True)
  indices_list = []
  for i in range(len(eval_dataframe)):
    indices_list.append(tuple([eval_dataframe["seq_starts"][i], eval_dataframe["seq_ends"][i]]))
  indices_list = merge_overlapping(indices_list)

  return eval_dataframe["id"][0], indices_list

In [0]:
# Load model
tokenizer = tokenizer_class.from_pretrained(args["output_dir"])
model = model_class.from_pretrained(args["output_dir"])
model.to(device)

# Specify the article folder to classify from
articles_folder = dev_articles
article_filenames = sorted(glob.glob(os.path.join(articles_folder, "*.txt")))

# Open a folder to write the labels to
labels_dir = '/../../../content/gdrive/My Drive/SI630/Project/results/train-dev/dev-SI'
if not os.path.exists(labels_dir):
  os.makedirs(labels_dir)

# For each article in the articles folder, do:
for i in range(len(article_filenames)):
  # Generate the tensor dataset
  eval_dataset, eval_dataframe = generate_SI_eval_dataset_from_article(article_filenames[i], tokenizer)

  # Classify!
  article_id, indices_list = classify_per_article(eval_dataframe, eval_dataset, model, tokenizer)
  
  # Write to the result file
  f = open(labels_dir + '/{}-SI.labels'.format(article_id), 'w')
  writer = csv.writer(f, delimiter='\t')
  for indices in indices_list:
    writer.writerow([article_id, indices[0], indices[1]])
  f.close()

INFO:transformers.tokenization_utils:Model name '/../../../content/gdrive/My Drive/SI630/Project/models/SI/train-dev' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming '/../../../content/gdrive/My Drive/SI630/Project/models/SI/train-dev' is a path, a model identifier, or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils:Didn't find file /../../../content/gdrive/My Drive/SI630/Project/models/SI/train-dev/added_tokens.json. We 

             id  seq_starts  seq_ends  label                           text
0     730081389           0         6      0                         police
1     730081389           0        10      0                     police had
2     730081389           0        21      0          police had previously
3     730081389           0        26      0     police had previously gone
4     730081389           0        29      0  police had previously gone to
...         ...         ...       ...    ...                            ...
6138  730081389        1734      1742      0                  ##point . ’ ’
6139  730081389        1743      1756      0                  advertisement
6140  730081389        1757      1767      0                     associated
6141  730081389        1757      1773      0               associated press
6142  730081389        1768      1773      0                          press

[6143 rows x 5 columns]
(6143, 5)


INFO:LOG:Creating TensorDataset from features dataframe
INFO:LOG:***** Running classification for article 730081389 *****
INFO:LOG:  Num sequences = 6143
INFO:LOG:  Batch size = 8
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


INFO:LOG:Generating evaluation dataset...


INFO:LOG:Creating features from dataframe


...

INFO:LOG:Creating TensorDataset from features dataframe
INFO:LOG:***** Running classification for article 999001323 *****
INFO:LOG:  Num sequences = 16999
INFO:LOG:  Batch size = 8
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


INFO:LOG:Generating evaluation dataset...


INFO:LOG:Creating features from dataframe


              id  seq_starts  seq_ends  label                    text
0      999001419           0         5      0                   trump
1      999001419           0        10      0              trump vows
2      999001419           0        13      0           trump vows to
3      999001419           0        18      0      trump vows to kick
4      999001419           0        22      0  trump vows to kick cnn
...          ...         ...       ...    ...                     ...
20885  999001419        5266      5268      0                      he
20886  999001419        5266      5273      0                 he said
20887  999001419        5266      5274      0               he said .
20888  999001419        5269      5273      0                    said
20889  999001419        5269      5274      0                  said .

[20890 rows x 5 columns]
(20890, 5)


INFO:LOG:Creating TensorDataset from features dataframe
INFO:LOG:***** Running classification for article 999001419 *****
INFO:LOG:  Num sequences = 20890
INFO:LOG:  Batch size = 8
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


In [0]:
# Make the labels into one big file
f_total = open("/../../../content/gdrive/My Drive/SI630/Project/results/train-dev/dev-SI-predictions.txt", 'w')
writer = csv.writer(f_total, delimiter='\t')
labels_filenames = sorted(glob.glob(os.path.join(labels_dir, "*.labels")))

# For each article in the labels folder, do:
for i in range(len(labels_filenames)):
  # Open the files
  f = open(labels_filenames[i], 'r')
  reader = csv.reader(f, delimiter='\t')
  for row in reader:
    writer.writerow(row)
  f.close()

f_total.close()

##Evaluation

In [0]:
# Score my predictions on dev articles for SI task against gold label, predicted with model trained on train articles
!python ./tools/task-SI_scorer.py -s "./../../../content/gdrive/My Drive/SI630/Project/results/train/dev-SI-predictions.labels" -r "./gold_labels/dev-labels-task1-span-identification"

# Baseline on Dev set: F1 = 0.01102 from https://propaganda.qcri.org/semeval2020-task11/leaderboard.php
# # Score my predictions on dev articles for SI task against gold label, predicted with model trained on train articles
# !python ./tools/task-SI_scorer.py -s "./baseline-output-SI.txt" -r "./gold_labels/dev-labels-task1-span-identification" -m

2020-04-25 19:22:03,388 - INFO - Checking user submitted file
2020-04-25 19:22:03,496 - INFO - Scoring the submission with precision and recall method
2020-04-25 19:22:04,420 - INFO - Precision=342.359941/3598=0.095153	Recall=930.153611/940=0.989525
2020-04-25 19:22:04,420 - INFO - F1=0.173611
2020-04-25 19:22:05,472 - INFO - Checking user submitted file
2020-04-25 19:22:05,484 - INFO - Scoring the submission with precision and recall method
2020-04-25 19:22:05,496 - INFO - Precision=7.408043/75=0.098774	Recall=3.834948/940=0.004080
2020-04-25 19:22:05,496 - INFO - F1=0.007836


#Task TC: Multi-Class Classification with BERT

##Preprocessing

In [0]:
# # Preprocess into tensor datasets (uncomment if files not generated, may need to change directory)
# train_dataset = generate_training_dataset_from_articles([train_articles], [train_TC_labels], tokenizer, task="TC")
# save_to_pickle(train_dataset, "/../../../content/gdrive/My Drive/SI630/Project/datasets/TC/train-train")

# train_dataset = generate_training_dataset_from_articles([train_articles, dev_articles], [train_TC_labels, dev_TC_labels], tokenizer, task="TC")
# save_to_pickle(train_dataset, "/../../../content/gdrive/My Drive/SI630/Project/datasets/TC/train-train-dev")

INFO:LOG:Generating training dataset...
INFO:LOG:Generating dataframe for folder datasets/train-articles


371 371


INFO:LOG:Creating features from dataframe


      label                                               text
0       0.0  The next transmission could be more pronounced...
1       0.0  when (the plague) comes again it starts from m...
2       5.0                                           appeared
3      10.0                             a very, very different
4       1.0  He also pointed to the presence of the pneumon...
...     ...                                                ...
6124   11.0                             the Left killed comedy
6125    6.0  no one looks in the mirror and thinks, ‘this b...
6126    9.0                                Columbia snowflakes
6127    6.0  Comrades, these jokes you have been listening ...
6128    6.0           I'm sure Patel felt very, like, accepted

[6129 rows x 2 columns]
(6129, 2)


INFO:LOG:Creating TensorDataset from features dataframe
INFO:LOG:Saved to /../../../content/gdrive/My Drive/SI630/Project/datasets/TC/train-train.pickle
INFO:LOG:Generating training dataset...
INFO:LOG:Generating dataframe for folder datasets/train-articles


371 371


INFO:LOG:Generating dataframe for folder datasets/dev-articles
INFO:LOG:Creating features from dataframe


75 75
      label                                               text
0       0.0  The next transmission could be more pronounced...
1       0.0  when (the plague) comes again it starts from m...
2       5.0                                           appeared
3      10.0                             a very, very different
4       1.0  He also pointed to the presence of the pneumon...
...     ...                                                ...
7187    9.0                            Trump-hating Republican
7188    8.0                                     grave hardship
7189    9.0                                  unbelievably rude
7190    9.0                                    wonderful woman
7191    7.0                                   our Constitution

[7192 rows x 2 columns]
(7192, 2)


INFO:LOG:Creating TensorDataset from features dataframe
INFO:LOG:Saved to /../../../content/gdrive/My Drive/SI630/Project/datasets/TC/train-train-dev.pickle


##Training

In [0]:
# Get the classes and pretrained config, tokenizer, and model with predetermined model
config_class, model_class, tokenizer_class = MODEL_CLASSES[args["model_type"]]
config = config_class.from_pretrained(args["model_name"], num_labels=len(PROP_TECH_TO_LABEL))
tokenizer = tokenizer_class.from_pretrained(args["model_name"])
model = model_class(config)
model.to(device)

# Load preprocessed tensor dataset
train_dataset = load_from_pickle("/../../../content/gdrive/My Drive/SI630/Project/datasets/TC/train-train-dev")
args['output_dir'] = "/../../../content/gdrive/My Drive/SI630/Project/models/TC/train-dev"

# Train the model with the dataset
global_step, tr_loss = train(train_dataset, model, tokenizer)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)
logger.info("Saving model checkpoint to %s", args['output_dir'])
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(args['output_dir'])
tokenizer.save_pretrained(args['output_dir'])
torch.save(args, os.path.join(args['output_dir'], 'training_args.bin'))

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 14,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11"

1.986183

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


1.183053

Epoch: 100%|██████████| 1/1 [02:04<00:00, 124.59s/it]
INFO:LOG: global_step = 899, average loss = 1.9034405332783837
INFO:LOG:Saving model checkpoint to /../../../content/gdrive/My Drive/SI630/Project/models/TC/train-dev


1.913615


INFO:transformers.configuration_utils:Configuration saved in /../../../content/gdrive/My Drive/SI630/Project/models/TC/train-dev/config.json
INFO:transformers.modeling_utils:Model weights saved in /../../../content/gdrive/My Drive/SI630/Project/models/TC/train-dev/pytorch_model.bin


##Classification & Evaluation

In [0]:
def classify_techniques(eval_dataframe, eval_dataset, model, tokenizer):
  """
  Classifies a single article dataset and returns article id with indices list
  """
  # Load the eval data and initialize sampler
  eval_sampler = SequentialSampler(eval_dataset)
  eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

  # Start Classification
  logger.info("***** Running classification for article {} *****".format(eval_dataframe["id"][0]))
  logger.info("  Num sequences = %d", len(eval_dataset))
  logger.info("  Batch size = %d", args['eval_batch_size'])
  preds = None

  # For each batch, evaluate
  for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2],
                'labels':         batch[3]}
      outputs = model(**inputs)
      logits = outputs[1]

    # Get predictions
    if preds is None:
      preds = logits.detach().cpu().numpy()
    else:
      preds = numpy.append(preds, logits.detach().cpu().numpy(), axis=0)

  # Get the most probable prediction
  preds = numpy.argmax(preds, axis=1)

  return preds

In [0]:
# Load model
args['output_dir'] = "/../../../content/gdrive/My Drive/SI630/Project/models/TC/train"
tokenizer = tokenizer_class.from_pretrained(args["output_dir"])
model = model_class.from_pretrained(args["output_dir"])
model.to(device)

# Specify the article folder to classify from
articles_folder = dev_articles
article_filenames = sorted(glob.glob(os.path.join(articles_folder, "*.txt")))

# Open a file to write the labels to
f = open('/../../../content/gdrive/My Drive/SI630/Project/results/train/dev-TC-predictions.txt', 'w')
writer = csv.writer(f, delimiter='\t')

# Generate the tensor dataset
eval_dataset, eval_dataframe = generate_TC_eval_dataset_from_article(dev_articles, dev_TC_template, tokenizer)

# Classify!
predictions = classify_techniques(eval_dataframe, eval_dataset, model, tokenizer)

# Write to the result file
for i in range(len(predictions)):
  writer.writerow([eval_dataframe["id"][i], LABEL_TO_PROP_TECH[predictions[i]], eval_dataframe["seq_starts"][i], eval_dataframe["seq_ends"][i]])

f.close()

# # Evaluate predictions
# with open(dev_TC_labels_file, "r") as f:
#   reader = csv.reader(f, delimiter="\t")
#   true_labels = []
#   for row in reader:
#     true_labels.append(PROP_TECH_TO_LABEL[row[1]])

# # Baseline
# with open("baseline-output-TC.txt", "r") as f:
#   reader = csv.reader(f, delimiter="\t")
#   base_line_preds_TC = []
#   for row in reader:
#     base_line_preds_TC.append(PROP_TECH_TO_LABEL[row[1]])

# Baseline on Dev set: F1 = 0.26529 from https://propaganda.qcri.org/semeval2020-task11/leaderboard.php
# my_score = f1_score(true_labels, predictions, labels=range(len(PROP_TECH_TO_LABEL)), average='micro')
# base_score = f1_score(base_line_preds_TC, predictions, labels=range(len(PROP_TECH_TO_LABEL)), average='micro')
# print("My f1 score with micro-average: {}".format(my_score))
# print("Baseline f1 score with micro-average: {}".format(base_score))

# print(Counter(predictions).most_common())
# print(Counter(true_labels).most_common())
# print(Counter(base_line_preds_TC).most_common())

INFO:transformers.tokenization_utils:Model name '/../../../content/gdrive/My Drive/SI630/Project/models/TC/train' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming '/../../../content/gdrive/My Drive/SI630/Project/models/TC/train' is a path, a model identifier, or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils:Didn't find file /../../../content/gdrive/My Drive/SI630/Project/models/TC/train/added_tokens.json. We won't load i

             id  ...                                               text
0     730093263  ...                                              white
1     730093263  ...                                              black
2     730093263  ...                            “true American heroes.”
3     730093263  ...                                              black
4     730093263  ...  If these two men had survived, and Quentin Lam...
...         ...  ...                                                ...
1058  999001419  ...                            Trump-hating Republican
1059  999001419  ...                                     grave hardship
1060  999001419  ...                                  unbelievably rude
1061  999001419  ...                                    wonderful woman
1062  999001419  ...                                   our Constitution

[1063 rows x 5 columns]
(1063, 5)


INFO:LOG:Creating TensorDataset from features dataframe
INFO:LOG:***** Running classification for article 730093263 *****
INFO:LOG:  Num sequences = 1063
INFO:LOG:  Batch size = 8
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()
